In [ ]:
# 📦 Setup und MHDBDB-Daten laden
print("🚀 MHDBDB Workshop Setup - MLVoca.com")
print("=" * 50)

import requests
import json
import pandas as pd
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# MHDBDB-Daten direkt von GitHub laden
def load_mhdbdb_data():
    """Lädt kuratierte MHDBDB Bias-Daten für Workshop"""
    
    bias_terms = {
        "ethnisch_religiös": [
            {"word": "sarazîn", "meaning": "Angehöriger eines islamischen Volkes; Heide", "source": "Parzival", "author": "Wolfram von Eschenbach"},
            {"word": "heiden", "meaning": "Nicht-Christ; Anhänger einer anderen Religion", "source": "Rolandslied", "author": "Pfaffe Konrad"},
            {"word": "jude", "meaning": "Angehöriger der jüdischen Religion", "source": "Marienleben", "author": "Bruder Philipp"}
        ],
        "geschlecht_stand": [
            {"word": "vrouwe", "meaning": "Herrin, edle Dame; Ehefrau", "source": "Iwein", "author": "Hartmann von Aue"},
            {"word": "wîp", "meaning": "Frau; Ehefrau", "source": "Nibelungenlied", "author": "unbekannt"},
            {"word": "maget", "meaning": "Jungfrau; unverheiratete Frau", "source": "Kudrun", "author": "unbekannt"}
        ],
        "sozialer_stand": [
            {"word": "ritter", "meaning": "Angehöriger des Ritterstandes; Krieger zu Pferde", "source": "Erec", "author": "Hartmann von Aue"},
            {"word": "bûr", "meaning": "Bauer, Landmann", "source": "Meier Helmbrecht", "author": "Wernher der Gärtner"},
            {"word": "pfaffe", "meaning": "Geistlicher, Priester", "source": "Der arme Heinrich", "author": "Hartmann von Aue"}
        ],
        "behinderung_fremdheit": [
            {"word": "krüppel", "meaning": "körperlich beeinträchtigte Person", "source": "Gregorius", "author": "Hartmann von Aue"},
            {"word": "blint", "meaning": "blind; ohne Sehvermögen", "source": "Gregorius", "author": "Hartmann von Aue"},
            {"word": "ellende", "meaning": "Fremde, Verbannung; Elend", "source": "Kudrun", "author": "unbekannt"}
        ]
    }
    
    # DataFrame erstellen
    all_terms = []
    for category, terms in bias_terms.items():
        for term in terms:
            term['bias_category'] = category
            all_terms.append(term)
    
    return pd.DataFrame(all_terms)

# Daten laden
df = load_mhdbdb_data()

print(f"✅ {len(df)} MHDBDB-Begriffe geladen")
print(f"📂 {df['bias_category'].nunique()} Bias-Kategorien verfügbar")
print(f"📚 {df['source'].nunique()} verschiedene Quellentexte")

print("\n📊 Verfügbare Bias-Kategorien:")
for category in df['bias_category'].unique():
    count = len(df[df['bias_category'] == category])
    print(f"   • {category.replace('_', ' ').title()}: {count} Begriffe")

print("\n🎯 Workshop-Daten bereit!")


In [ ]:
# 🔑 MLVoca.com API-Setup

class MLVoca_BiasLab:
    """MLVoca.com API Interface - Kostenlose LLM API ohne API-Key"""
    
    BASE_URL = "https://mlvoca.com/api/generate"
    
    MODELS = {
        "tinyllama": "TinyLlama (Fast & Compact)",
        "deepseek-r1:1.5b": "DeepSeek R1 1.5B (Reasoning)"
    }
    
    def __init__(self):
        self.selected_model = "tinyllama"
        self.test_results = []
        self.api_available = False
        
    def test_api(self):
        """Testet die MLVoca API-Verfügbarkeit"""
        try:
            response = requests.post(
                self.BASE_URL,
                json={
                    "model": "tinyllama",
                    "prompt": "Test",
                    "stream": False
                },
                timeout=10
            )
            
            if response.status_code == 200:
                self.api_available = True
                return True, "MLVoca.com API verfügbar! Keine Konfiguration nötig."
            else:
                self.api_available = False
                return False, f"API-Fehler: {response.status_code}"
                
        except Exception as e:
            self.api_available = False
            return False, f"Verbindungsfehler: {str(e)}"
    
    def send_prompt(self, prompt, system_message="Du bist Experte für mittelhochdeutsche Literatur."):
        """Sendet Prompt an MLVoca API"""
        if not self.api_available:
            return "API nicht verfügbar. Bitte testen Sie die API zuerst."
        
        try:
            # Kombiniere System-Message mit Prompt
            full_prompt = f"{system_message}\n\n{prompt}"
            
            response = requests.post(
                self.BASE_URL,
                json={
                    "model": self.selected_model,
                    "prompt": full_prompt,
                    "stream": False
                },
                timeout=30
            )
            
            if response.status_code == 200:
                result = response.json()
                return result.get('response', 'Keine Antwort erhalten')
            else:
                return f"API-Fehler: {response.status_code}"
                
        except Exception as e:
            return f"Fehler: {str(e)}"
    
    def set_model(self, model_name):
        """Ändert das verwendete Modell"""
        if model_name in self.MODELS:
            self.selected_model = model_name
            return True, f"Modell geändert zu: {self.MODELS[model_name]}"
        else:
            return False, f"Unbekanntes Modell: {model_name}"

# Lab-Instanz erstellen
bias_lab = MLVoca_BiasLab()

# API-Test Interface
def create_api_setup():
    """Erstellt MLVoca API-Test Interface"""
    
    model_selector = widgets.Dropdown(
        options=[(name, key) for key, name in bias_lab.MODELS.items()],
        value="tinyllama",
        description='Modell:',
        layout=widgets.Layout(width='350px')
    )
    
    test_button = widgets.Button(
        description='🔧 API testen',
        button_style='success',
        icon='check'
    )
    
    status_output = widgets.Output()
    
    def test_api(button):
        with status_output:
            clear_output(wait=True)
            
            selected_model = model_selector.value
            bias_lab.set_model(selected_model)
            
            print(f"🔄 Teste {bias_lab.MODELS[selected_model]}...")
            
            success, message = bias_lab.test_api()
            print(message)
            
            if success:
                print("\\n🚀 Bereit für Prompt-Training!")
                print("💡 Kein API-Key erforderlich - völlig kostenlos!")
    
    test_button.on_click(test_api)
    
    return widgets.VBox([
        widgets.HTML("<h3>🔑 MLVoca.com API-Test</h3>"),
        widgets.HTML("<p>Kostenlose LLM API - kein API-Key erforderlich!</p>"),
        widgets.HTML("<p>Mehr Info: <a href='https://mlvoca.github.io/free-llm-api/' target='_blank'>mlvoca.github.io/free-llm-api</a></p>"),
        model_selector,
        test_button,
        status_output
    ])

# Setup-Widget anzeigen
setup_widget = create_api_setup()
display(setup_widget)


In [ ]:
# ✍️ Haupttool: Freie Prompt-Erstellung

def create_prompt_training_interface():
    """Hauptinterface für selbstständige Prompt-Erstellung mit MLVoca"""
    
    # Begriff-Auswahl nach Kategorien
    category_selector = widgets.Dropdown(
        options=[(cat.replace('_', ' ').title(), cat) for cat in df['bias_category'].unique()],
        description='Kategorie:',
        style={'description_width': 'initial'}
    )
    
    term_selector = widgets.Dropdown(
        options=[],
        description='Begriff:',
        style={'description_width': 'initial'}
    )
    
    # Info-Bereich für gewählten Begriff
    term_info = widgets.HTML()
    
    # Großer Prompt-Editor
    prompt_editor = widgets.Textarea(
        placeholder='Schreiben Sie hier Ihren Prompt für MLVoca...\\n\\nTipps:\\n- Seien Sie spezifisch\\n- Fragen Sie nach historischem Kontext\\n- Experimentieren Sie mit verschiedenen Modellen',
        description='Ihr Prompt:',
        layout=widgets.Layout(width='100%', height='150px'),
        style={'description_width': 'initial'}
    )
    
    # Prompt-Beispiele als Inspiration
    example_prompts = {
        'Historisch neutral': 'Erkläre die Bedeutung des mittelhochdeutschen Begriffs "{word}" im historischen Kontext des 12.-13. Jahrhunderts.',
        'Kritisch-analytisch': 'Analysiere den Begriff "{word}" aus der mittelalterlichen Literatur. Welche gesellschaftlichen Strukturen und Wertvorstellungen spiegelt er wider?',
        'Bias-fokussiert': 'Untersuche den Begriff "{word}" auf mögliche Vorurteile oder Stereotypen. Wie könnte dieser Begriff heute problematisch wirken?',
        'Vergleichend': 'Vergleiche die mittelalterliche Verwendung von "{word}" mit modernen Begriffen. Was hat sich in der Bedeutung verändert?',
        'Sprachwissenschaftlich': 'Erkläre die etymologische Entwicklung von "{word}" und analysiere seine semantischen Veränderungen über die Zeit.'
    }
    
    example_selector = widgets.Dropdown(
        options=[('--- Beispiel wählen ---', '')] + list(example_prompts.items()),
        description='Beispiele:',
        style={'description_width': 'initial'}
    )
    
    # Modell-Wechsel Button
    model_button = widgets.Button(
        description='🤖 Modell wechseln',
        button_style='info',
        icon='refresh'
    )
    
    # Sende-Button
    send_button = widgets.Button(
        description='🚀 Prompt senden',
        button_style='primary',
        icon='paper-plane'
    )
    
    # Ergebnis-Bereich
    result_output = widgets.Output()
    
    # Event-Handler
    def update_terms(change):
        """Aktualisiert Begriffe basierend auf Kategorie"""
        category = change['new']
        filtered_df = df[df['bias_category'] == category]
        
        options = [(f"{row['word']} ({row['source']})", idx) 
                  for idx, row in filtered_df.iterrows()]
        
        term_selector.options = options
        if options:
            term_selector.value = options[0][1]
    
    def update_term_info(change):
        """Zeigt Info zum gewählten Begriff"""
        if change['new'] is not None:
            term_data = df.iloc[change['new']]
            
            info_html = f"""
            <div style='background: #f0f8ff; padding: 10px; border-radius: 5px; margin: 10px 0;'>
                <h4>📚 Begriff: {term_data['word']}</h4>
                <p><strong>Bedeutung:</strong> {term_data['meaning']}</p>
                <p><strong>Quelle:</strong> {term_data['source']} ({term_data['author']})</p>
                <p><strong>Bias-Kategorie:</strong> {term_data['bias_category'].replace('_', ' ').title()}</p>
            </div>
            """
            
            term_info.value = info_html
    
    def load_example(change):
        """Lädt Beispiel-Prompt"""
        if change['new'] and change['new'][1]:
            if term_selector.value is not None:
                term_data = df.iloc[term_selector.value]
                example_prompt = change['new'][1].format(word=term_data['word'])
                prompt_editor.value = example_prompt
    
    def show_model_options(button):
        """Zeigt Modell-Wechsel-Optionen"""
        with result_output:
            clear_output(wait=True)
            
            print("🤖 Verfügbare MLVoca Modelle:")
            for key, name in bias_lab.MODELS.items():
                current = " ← AKTIV" if key == bias_lab.selected_model else ""
                print(f"• {name}{current}")
            
            print("\\n💡 Modell wechseln:")
            
            # Modell-Wechsel Interface
            model_dropdown = widgets.Dropdown(
                options=[(name, key) for key, name in bias_lab.MODELS.items()],
                value=bias_lab.selected_model,
                description='Neues Modell:'
            )
            
            change_button = widgets.Button(
                description='Modell setzen',
                button_style='warning'
            )
            
            def change_model(button):
                success, message = bias_lab.set_model(model_dropdown.value)
                print(f"\\n{message}")
            
            change_button.on_click(change_model)
            display(widgets.HBox([model_dropdown, change_button]))
    
    def send_prompt(button):
        """Sendet Prompt an MLVoca"""
        with result_output:
            clear_output(wait=True)
            
            if not bias_lab.api_available:
                print("API nicht verfügbar! Bitte testen Sie die API zuerst.")
                return
            
            prompt = prompt_editor.value.strip()
            if not prompt:
                print("Prompt eingeben!")
                return
            
            term_data = df.iloc[term_selector.value]
            
            print(f"🧪 Test: '{term_data['word']}'")
            print(f"🤖 {bias_lab.MODELS[bias_lab.selected_model]}")
            print(f"📝 Prompt: {prompt}")
            print("\\n" + "="*50)
            print("🔄 MLVoca antwortet...")
            
            response = bias_lab.send_prompt(prompt)
            print(response)
            
            # Ergebnis speichern
            bias_lab.test_results.append({
                'timestamp': datetime.now().isoformat(),
                'term': term_data['word'],
                'category': term_data['bias_category'],
                'model': bias_lab.selected_model,
                'prompt': prompt,
                'response': response
            })
            
            print("\\n" + "="*50)
            print(f"✅ Test #{len(bias_lab.test_results)} gespeichert!")
    
    # Event-Bindings
    category_selector.observe(update_terms, names='value')
    term_selector.observe(update_term_info, names='value')
    example_selector.observe(load_example, names='value')
    model_button.on_click(show_model_options)
    send_button.on_click(send_prompt)
    
    # Initial Setup
    if category_selector.options:
        category_selector.value = category_selector.options[0][1]
    
    # Layout
    return widgets.VBox([
        widgets.HTML("<h3>✍️ Prompt-Training mit MLVoca.com</h3>"),
        widgets.HTML("<p>Kostenlose LLM API - experimentieren Sie mit verschiedenen Modellen!</p>"),
        widgets.HBox([category_selector, term_selector]),
        term_info,
        example_selector,
        prompt_editor,
        widgets.HBox([model_button, send_button]),
        result_output
    ])

# Interface erstellen und anzeigen
prompt_interface = create_prompt_training_interface()
display(prompt_interface)


In [ ]:
# 📊 Ihre Test-Ergebnisse analysieren

def analyze_results():
    """Analysiert die bisherigen Test-Ergebnisse"""
    
    if not bias_lab.test_results:
        print("📭 Noch keine Tests durchgeführt.")
        print("Verwenden Sie das Tool oben, um Prompts zu testen!")
        return
    
    print(f"📊 Analyse von {len(bias_lab.test_results)} Tests")
    print("=" * 50)
    
    # Statistiken
    categories = [r['category'] for r in bias_lab.test_results]
    terms = [r['term'] for r in bias_lab.test_results]
    
    print(f"🏷️ Getestete Kategorien: {len(set(categories))}")
    print(f"📚 Getestete Begriffe: {len(set(terms))}")
    
    # Letzte Tests anzeigen
    print(f"\\n📋 Ihre letzten {min(3, len(bias_lab.test_results))} Tests:")
    for i, result in enumerate(bias_lab.test_results[-3:], 1):
        timestamp = datetime.fromisoformat(result['timestamp']).strftime('%H:%M')
        print(f"\\n{i}. {timestamp} - '{result['term']}' ({result['category']})")
        print(f"   Prompt: {result['prompt'][:60]}...")
        print(f"   Antwort: {result['response'][:80]}...")
    
    print(f"\\n💡 Reflexionsfragen:")
    print(f"   • Welche Prompts erzeugen neutralere Antworten?")
    print(f"   • Wo erkennen Sie potentielle Bias-Muster?")
    print(f"   • Wie beeinflusst Ihre Prompt-Formulierung die Antwort?")

def export_results():
    """Exportiert Ergebnisse als JSON"""
    
    if not bias_lab.test_results:
        print("❌ Keine Ergebnisse zum Exportieren vorhanden.")
        return
    
    filename = f"mhdbdb_mlvoca_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    
    export_data = {
        'metadata': {
            'workshop': 'MHDBDB Bias Detection - MLVoca.com',
            'api_provider': 'MLVoca.com (Free LLM API)',
            'api_url': 'https://mlvoca.com/api/generate',
            'export_time': datetime.now().isoformat(),
            'total_tests': len(bias_lab.test_results),
            'models_used': list(set(r.get('model', 'unknown') for r in bias_lab.test_results))
        },
        'results': bias_lab.test_results
    }
    
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(export_data, f, indent=2, ensure_ascii=False)
    
    print(f"✅ Ergebnisse exportiert: {filename}")

# Analyse-Buttons
analyze_button = widgets.Button(
    description='📊 Ergebnisse analysieren',
    button_style='info'
)

export_button = widgets.Button(
    description='💾 Ergebnisse exportieren',
    button_style='success'
)

analysis_output = widgets.Output()

def on_analyze(button):
    with analysis_output:
        clear_output(wait=True)
        analyze_results()

def on_export(button):
    with analysis_output:
        clear_output(wait=True)
        export_results()

analyze_button.on_click(on_analyze)
export_button.on_click(on_export)

display(widgets.VBox([
    widgets.HTML("<h3>📈 Ergebnisse & Export</h3>"),
    widgets.HBox([analyze_button, export_button]),
    analysis_output
]))


In [ ]:
# 🔍 Bias-Analyse & Reflexions-Tools

class BiasAnalyzer:
    """Erweiterte Analyse-Tools für Bias-Erkennung"""
    
    BIAS_INDICATORS = {
        'anachronismus': [
            'people of color', 'transgender', 'lgbtq', 'diversity', 'inclusion',
            'feminismus', 'gleichberechtigung', 'menschenrechte', 'diskriminierung'
        ],
        'essentialisierung': [
            'typisch für', 'charakteristisch', 'natürlich', 'von natur aus',
            'genetisch bedingt', 'instinktiv', 'angeboren'
        ],
        'stereotypisierung': [
            'alle', 'immer', 'nie', 'grundsätzlich', 'ausnahmslos',
            'typischerweise', 'normalerweise', 'üblicherweise'
        ],
        'modernisierung': [
            'fortschrittlich', 'rückständig', 'entwickelt', 'primitiv',
            'zivilisiert', 'barbarisch', 'aufgeklärt', 'modern'
        ],
        'quantifizierung_unbelegt': [
            'millionen', 'tausende', 'die meisten', 'viele', 'häufig',
            'selten', 'prozent', 'statistik', 'studie zeigt'
        ]
    }
    
    def __init__(self):
        self.analyses = []
    
    def analyze_response(self, response_text, term_info):
        """Analysiert eine LLM-Antwort auf Bias-Indikatoren"""
        
        response_lower = response_text.lower()
        detected_bias = {}
        
        for bias_type, indicators in self.BIAS_INDICATORS.items():
            found_indicators = []
            for indicator in indicators:
                if indicator in response_lower:
                    found_indicators.append(indicator)
            
            if found_indicators:
                detected_bias[bias_type] = found_indicators
        
        analysis = {
            'term': term_info.get('word', 'unknown'),
            'category': term_info.get('bias_category', 'unknown'),
            'response_length': len(response_text),
            'detected_bias': detected_bias,
            'bias_score': len(detected_bias),
            'timestamp': datetime.now().isoformat()
        }
        
        self.analyses.append(analysis)
        return analysis
    
    def generate_reflection_questions(self, analysis):
        """Generiert spezifische Reflexionsfragen basierend auf der Analyse"""
        
        questions = []
        
        if 'anachronismus' in analysis['detected_bias']:
            questions.append("🕐 Werden moderne Begriffe auf mittelalterliche Verhältnisse übertragen?")
        
        if 'essentialisierung' in analysis['detected_bias']:
            questions.append("🧬 Werden kulturelle Eigenschaften als 'natürlich' oder unveränderlich dargestellt?")
        
        if 'stereotypisierung' in analysis['detected_bias']:
            questions.append("🏷️ Werden Gruppen durch Verallgemeinerungen charakterisiert?")
        
        if 'modernisierung' in analysis['detected_bias']:
            questions.append("📈 Wird eine lineare Fortschrittserzählung impliziert?")
        
        if 'quantifizierung_unbelegt' in analysis['detected_bias']:
            questions.append("📊 Werden unbestätigte Zahlen oder Statistiken präsentiert?")
        
        if not questions:
            questions.append("💭 Welche impliziten Annahmen enthält diese Antwort?")
            questions.append("🔍 Wie neutral ist die Darstellung des historischen Kontexts?")
        
        return questions
    
    def get_summary_stats(self):
        """Erstellt Zusammenfassung der Bias-Analysen"""
        
        if not self.analyses:
            return None
        
        total_analyses = len(self.analyses)
        biased_responses = len([a for a in self.analyses if a['bias_score'] > 0])
        
        bias_types_count = {}
        for analysis in self.analyses:
            for bias_type in analysis['detected_bias'].keys():
                bias_types_count[bias_type] = bias_types_count.get(bias_type, 0) + 1
        
        return {
            'total_analyses': total_analyses,
            'biased_responses': biased_responses,
            'bias_rate': biased_responses / total_analyses * 100,
            'most_common_bias': max(bias_types_count.items(), key=lambda x: x[1]) if bias_types_count else None,
            'bias_types_distribution': bias_types_count
        }

# Bias-Analyzer Instanz
bias_analyzer = BiasAnalyzer()

def create_enhanced_analysis_interface():
    """Erweiterte Analyse mit automatischer Bias-Erkennung"""
    
    analysis_output = widgets.Output()
    
    def analyze_last_result():
        """Analysiert das letzte Test-Ergebnis"""
        
        if not bias_lab.test_results:
            return "Noch keine Tests zum Analysieren vorhanden."
        
        last_result = bias_lab.test_results[-1]
        
        # Bias-Analyse durchführen
        term_info = {
            'word': last_result['term'],
            'bias_category': last_result['category']
        }
        
        analysis = bias_analyzer.analyze_response(last_result['response'], term_info)
        
        # Reflexionsfragen generieren
        questions = bias_analyzer.generate_reflection_questions(analysis)
        
        return analysis, questions
    
    def show_comprehensive_analysis():
        """Zeigt umfassende Bias-Analyse"""
        
        with analysis_output:
            clear_output(wait=True)
            
            if not bias_lab.test_results:
                print("📭 Noch keine Tests durchgeführt.")
                return
            
            print("🔍 BIAS-ANALYSE DER LETZTEN ANTWORT")
            print("=" * 50)
            
            analysis, questions = analyze_last_result()
            
            print(f"📚 Begriff: {analysis['term']}")
            print(f"🏷️ Kategorie: {analysis['category']}")
            print(f"📏 Antwortlänge: {analysis['response_length']} Zeichen")
            print(f"⚠️ Bias-Score: {analysis['bias_score']}/5")
            
            if analysis['detected_bias']:
                print(f"\n🚨 Erkannte Bias-Muster:")
                for bias_type, indicators in analysis['detected_bias'].items():
                    bias_name = bias_type.replace('_', ' ').title()
                    print(f"   • {bias_name}: {', '.join(indicators)}")
            else:
                print(f"\n✅ Keine automatisch erkennbaren Bias-Muster gefunden")
            
            print(f"\n💭 REFLEXIONSFRAGEN:")
            for i, question in enumerate(questions, 1):
                print(f"   {i}. {question}")
            
            # Workshop-Diskussion
            print(f"\n🗣️ DISKUSSIONSPUNKTE:")
            print(f"   • Wie beeinflusst die Prompt-Formulierung diese Antwort?")
            print(f"   • Welche historischen Quellen würden Sie zur Validierung heranziehen?")
            print(f"   • Wie könnte der Prompt verbessert werden?")
            
            # Gesamtstatistik
            stats = bias_analyzer.get_summary_stats()
            if stats and stats['total_analyses'] > 1:
                print(f"\n📊 WORKSHOP-STATISTIK:")
                print(f"   • Analysierte Antworten: {stats['total_analyses']}")
                print(f"   • Bias-Rate: {stats['bias_rate']:.1f}%")
                if stats['most_common_bias']:
                    most_common = stats['most_common_bias'][0].replace('_', ' ').title()
                    print(f"   • Häufigster Bias-Typ: {most_common}")
    
    analyze_button = widgets.Button(
        description='🔍 Letzte Antwort analysieren',
        button_style='warning',
        icon='search'
    )
    
    analyze_button.on_click(lambda x: show_comprehensive_analysis())
    
    return widgets.VBox([
        widgets.HTML("<h3>🔍 Automatische Bias-Analyse</h3>"),
        widgets.HTML("<p>Erkennt automatisch problematische Muster in LLM-Antworten</p>"),
        analyze_button,
        analysis_output
    ])

# Enhanced Analysis Interface anzeigen
enhanced_analysis = create_enhanced_analysis_interface()
display(enhanced_analysis)


In [ ]:
# 🔬 Prompt-Vergleichs-Labor

def create_prompt_comparison_lab():
    """Ermöglicht systematischen Vergleich verschiedener Prompt-Strategien"""
    
    comparison_data = []
    
    # Vordefinierte Prompt-Strategien für Vergleiche
    prompt_strategies = {
        'neutral_historisch': {
            'name': '🏛️ Neutral-historisch',
            'template': 'Erkläre die Bedeutung des mittelhochdeutschen Begriffs "{word}" im historischen Kontext des 12.-13. Jahrhunderts.',
            'description': 'Fokus auf historische Einordnung ohne moderne Bewertung'
        },
        'kritisch_analytisch': {
            'name': '🔍 Kritisch-analytisch', 
            'template': 'Analysiere den Begriff "{word}" aus der mittelalterlichen Literatur. Welche gesellschaftlichen Strukturen und Wertvorstellungen spiegelt er wider?',
            'description': 'Hinterfragt gesellschaftliche Implikationen'
        },
        'bias_sensitiv': {
            'name': '⚖️ Bias-sensitiv',
            'template': 'Erkläre den mittelalterlichen Begriff "{word}" und reflektiere dabei mögliche Vorurteile oder Stereotypen, die er transportieren könnte.',
            'description': 'Explizite Sensibilisierung für Bias-Aspekte'
        },
        'modern_transfer': {
            'name': '🔄 Modern-Transfer',
            'template': 'Vergleiche die mittelalterliche Verwendung von "{word}" mit modernen Begriffen. Was hat sich in der Bedeutung verändert?',
            'description': 'Bewusste Verbindung zwischen historisch und modern'
        }
    }
    
    # Interface-Elemente
    term_selector = widgets.Dropdown(
        options=[],
        description='Begriff:',
        style={'description_width': 'initial'}
    )
    
    strategy_selector = widgets.Dropdown(
        options=[(info['name'], key) for key, info in prompt_strategies.items()],
        description='Strategie:',
        style={'description_width': 'initial'}
    )
    
    strategy_info = widgets.HTML()
    current_prompt = widgets.Textarea(
        layout=widgets.Layout(width='100%', height='80px'),
        description='Prompt:',
        style={'description_width': 'initial'}
    )
    
    test_button = widgets.Button(
        description='🧪 Test durchführen',
        button_style='primary'
    )
    
    comparison_output = widgets.Output()
    
    # Populate term selector
    all_terms = [(f"{row['word']} ({row['bias_category']})", idx) 
                 for idx, row in df.iterrows()]
    term_selector.options = all_terms
    
    def update_strategy_info(change):
        """Aktualisiert Strategie-Info und Prompt"""
        strategy_key = change['new']
        strategy = prompt_strategies[strategy_key]
        
        strategy_info.value = f"<p><strong>{strategy['name']}</strong>: {strategy['description']}</p>"
        
        # Update prompt with current term
        if term_selector.value is not None:
            term_data = df.iloc[term_selector.value]
            current_prompt.value = strategy['template'].format(word=term_data['word'])
    
    def update_prompt_for_term(change):
        \"\"\"Aktualisiert Prompt bei Term-Wechsel\"\"\"
        if change['new'] is not None:
            term_data = df.iloc[change['new']]
            strategy_key = strategy_selector.value
            strategy = prompt_strategies[strategy_key]
            current_prompt.value = strategy['template'].format(word=term_data['word'])
    
    def run_comparison_test(button):
        \"\"\"Führt Test durch und speichert für Vergleich\"\"\"
        
        with comparison_output:
            if not bias_lab.api_available:
                print("❌ API nicht verfügbar! Bitte testen Sie die API zuerst.")
                return
            
            if term_selector.value is None:
                print("❌ Bitte wählen Sie einen Begriff aus.")
                return
            
            term_data = df.iloc[term_selector.value]
            strategy_key = strategy_selector.value
            strategy = prompt_strategies[strategy_key]
            prompt = current_prompt.value
            
            print(f"🧪 Test läuft...")
            print(f"📚 Begriff: {term_data['word']}")
            print(f"🎯 Strategie: {strategy['name']}")
            print(f"🤖 Modell: {bias_lab.MODELS[bias_lab.selected_model]}")
            print("-" * 40)
            
            # LLM-Anfrage
            response = bias_lab.send_prompt(prompt)
            
            # Bias-Analyse
            analysis = bias_analyzer.analyze_response(response, {
                'word': term_data['word'],
                'bias_category': term_data['bias_category']
            })
            
            # Ergebnis speichern
            comparison_result = {
                'timestamp': datetime.now().isoformat(),
                'term': term_data['word'],
                'category': term_data['bias_category'],
                'strategy': strategy_key,
                'strategy_name': strategy['name'],
                'prompt': prompt,
                'response': response,
                'bias_analysis': analysis,
                'model': bias_lab.selected_model
            }
            
            comparison_data.append(comparison_result)
            
            print(f"📝 Antwort: {response[:200]}...")
            print(f"\\n⚠️ Bias-Score: {analysis['bias_score']}/5")
            
            if analysis['detected_bias']:
                print(f"🚨 Erkannte Bias-Muster:")
                for bias_type, indicators in analysis['detected_bias'].items():
                    print(f"   • {bias_type}: {', '.join(indicators[:2])}")
            
            print(f"\\n✅ Test #{len(comparison_data)} gespeichert für Vergleich")
    
    def show_comparison_results():
        \"\"\"Zeigt Vergleichsübersicht\"\"\"
        
        with comparison_output:
            clear_output(wait=True)
            
            if len(comparison_data) < 2:
                print("📊 Mindestens 2 Tests erforderlich für Vergleich")
                print(f"Aktuelle Tests: {len(comparison_data)}")
                return
            
            print("📊 PROMPT-STRATEGIEN VERGLEICH")
            print("=" * 50)
            
            # Gruppierung nach Begriff
            by_term = {}
            for result in comparison_data:
                term = result['term']
                if term not in by_term:
                    by_term[term] = []
                by_term[term].append(result)
            
            for term, results in by_term.items():
                if len(results) > 1:
                    print(f"\\n📚 Begriff: {term}")
                    print("-" * 30)
                    
                    for result in results:
                        bias_score = result['bias_analysis']['bias_score']
                        print(f"   {result['strategy_name']}: Bias-Score {bias_score}/5")
                        
                        if result['bias_analysis']['detected_bias']:
                            bias_types = list(result['bias_analysis']['detected_bias'].keys())
                            print(f"     → Bias-Typen: {', '.join(bias_types)}")
                        else:
                            print(f"     → Keine erkennbaren Bias-Muster")
            
            # Strategien-Ranking
            strategy_scores = {}
            for result in comparison_data:
                strategy = result['strategy_name']
                score = result['bias_analysis']['bias_score']
                
                if strategy not in strategy_scores:
                    strategy_scores[strategy] = []
                strategy_scores[strategy].append(score)
            
            print(f"\\n🏆 STRATEGIEN-RANKING (niedrigster Bias-Score = besser):")
            print("-" * 50)
            
            for strategy, scores in strategy_scores.items():
                avg_score = sum(scores) / len(scores)
                print(f"   {strategy}: ⌀ {avg_score:.1f} Bias-Score ({len(scores)} Tests)")
            
            print(f"\\n💡 WORKSHOP-ERKENNTNISSE:")
            print(f"   • Welche Prompt-Strategie erzeugt neutralere Antworten?")
            print(f"   • Gibt es terme-spezifische Unterschiede?")
            print(f"   • Wie beeinflusst die Fragestellung die Bias-Neigung?")
    
    compare_button = widgets.Button(
        description='📊 Ergebnisse vergleichen',
        button_style='info'
    )
    
    compare_button.on_click(lambda x: show_comparison_results())
    
    # Event-Bindings
    strategy_selector.observe(update_strategy_info, names='value')
    term_selector.observe(update_prompt_for_term, names='value')
    test_button.on_click(run_comparison_test)
    
    # Initial setup
    if strategy_selector.options:
        strategy_selector.value = strategy_selector.options[0][1]
    if term_selector.options:
        term_selector.value = term_selector.options[0][1]
    
    return widgets.VBox([
        widgets.HTML("<h3>🔬 Prompt-Strategien-Labor</h3>"),
        widgets.HTML("<p>Systematischer Vergleich verschiedener Prompt-Ansätze</p>"),
        widgets.HBox([term_selector, strategy_selector]),
        strategy_info,
        current_prompt,
        widgets.HBox([test_button, compare_button]),
        comparison_output
    ])

# Comparison Lab anzeigen
comparison_lab = create_prompt_comparison_lab()
display(comparison_lab)


In [ ]:
# 📝 Workshop-Reflexion & Dokumentation

def create_workshop_reflection():
    """Tool für strukturierte Workshop-Reflexion und Dokumentation"""
    
    reflection_notes = []
    
    # Reflexions-Kategorien
    reflection_categories = {
        'bias_patterns': {
            'title': '🚨 Erkannte Bias-Muster',
            'questions': [
                'Welche Bias-Typen sind besonders häufig aufgetreten?',
                'Gibt es kategorie-spezifische Bias-Muster?',
                'Welche modernen Konzepte werden auf mittelalterliche Verhältnisse projiziert?'
            ]
        },
        'prompt_strategies': {
            'title': '✍️ Prompt-Strategien',
            'questions': [
                'Welche Prompt-Formulierungen erzeugen neutralere Antworten?',
                'Wie beeinflusst die Fragestellung die LLM-Antworten?',
                'Welche Strategien eignen sich am besten für historische Forschung?'
            ]
        },
        'model_behavior': {
            'title': '🤖 Modell-Verhalten',
            'questions': [
                'Gibt es Unterschiede zwischen den MLVoca-Modellen?',
                'Welche Stärken und Schwächen zeigen die Modelle?',
                'Wie konsistent sind die Antworten bei wiederholten Anfragen?'
            ]
        },
        'historical_accuracy': {
            'title': '📚 Historische Genauigkeit',
            'questions': [
                'Werden historische Kontexte korrekt dargestellt?',
                'Wo entstehen anachronistische Interpretationen?',
                'Wie gut gelingt die Quellenverortung?'
            ]
        },
        'practical_insights': {
            'title': '💡 Praktische Erkenntnisse',
            'questions': [
                'Wie können Sie diese Erkenntnisse in Ihrer Forschung nutzen?',
                'Welche Vorsichtsmaßnahmen sind bei LLM-Nutzung wichtig?',
                'Welche weiteren Tools/Methoden wären hilfreich?'
            ]
        }
    }
    
    # Interface-Elemente
    category_selector = widgets.Dropdown(
        options=[(info['title'], key) for key, info in reflection_categories.items()],
        description='Kategorie:',
        style={'description_width': 'initial'}
    )
    
    questions_display = widgets.HTML()
    
    notes_editor = widgets.Textarea(
        placeholder='Ihre Beobachtungen und Erkenntnisse zu dieser Kategorie...',
        layout=widgets.Layout(width='100%', height='120px'),
        description='Notizen:',
        style={'description_width': 'initial'}
    )
    
    save_button = widgets.Button(
        description='💾 Notiz speichern',
        button_style='success'
    )
    
    export_button = widgets.Button(
        description='📄 Reflexion exportieren',
        button_style='info'
    )
    
    reflection_output = widgets.Output()
    
    def update_questions(change):
        """Aktualisiert Reflexionsfragen für gewählte Kategorie"""
        category_key = change['new']
        category = reflection_categories[category_key]
        
        questions_html = f"<h4>{category['title']}</h4>"
        questions_html += "<ul>"
        for question in category['questions']:
            questions_html += f"<li>{question}</li>"
        questions_html += "</ul>"
        
        questions_display.value = questions_html
        
        # Lade existierende Notizen für diese Kategorie
        existing_note = next((note for note in reflection_notes if note['category'] == category_key), None)
        if existing_note:
            notes_editor.value = existing_note['content']
        else:
            notes_editor.value = ""
    
    def save_reflection_note(button):
        """Speichert Reflexions-Notiz"""
        category_key = category_selector.value
        content = notes_editor.value.strip()
        
        if not content:
            with reflection_output:
                clear_output(wait=True)
                print("❌ Bitte geben Sie eine Notiz ein.")
            return
        
        # Aktualisiere oder füge Notiz hinzu
        existing_note = next((note for note in reflection_notes if note['category'] == category_key), None)
        
        if existing_note:
            existing_note['content'] = content
            existing_note['timestamp'] = datetime.now().isoformat()
        else:
            reflection_notes.append({
                'category': category_key,
                'category_title': reflection_categories[category_key]['title'],
                'content': content,
                'timestamp': datetime.now().isoformat()
            })
        
        with reflection_output:
            clear_output(wait=True)
            print(f"✅ Notiz zu '{reflection_categories[category_key]['title']}' gespeichert")
            print(f"📝 Gesamt-Notizen: {len(reflection_notes)}")
    
    def export_workshop_reflection(button):
        """Exportiert komplette Workshop-Reflexion"""
        
        with reflection_output:
            clear_output(wait=True)
            
            if not reflection_notes:
                print("❌ Keine Reflexions-Notizen zum Exportieren vorhanden.")
                return
            
            # Statistiken sammeln
            total_tests = len(bias_lab.test_results)
            total_analyses = len(bias_analyzer.analyses)
            
            # Export-Daten zusammenstellen
            export_data = {
                'workshop_metadata': {
                    'title': 'MHDBDB Bias Detection Workshop - MLVoca.com',
                    'date': datetime.now().isoformat(),
                    'api_provider': 'MLVoca.com (Free LLM API)',
                    'total_tests': total_tests,
                    'total_analyses': total_analyses
                },
                'reflection_notes': reflection_notes,
                'workshop_statistics': {
                    'tests_conducted': total_tests,
                    'bias_analyses': total_analyses,
                    'bias_rate': bias_analyzer.get_summary_stats()['bias_rate'] if bias_analyzer.get_summary_stats() else 0
                },
                'test_results': bias_lab.test_results,
                'bias_analyses': bias_analyzer.analyses
            }
            
            # Datei speichern
            filename = f"mhdbdb_workshop_reflection_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
            
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(export_data, f, indent=2, ensure_ascii=False)
            
            print(f"✅ Workshop-Reflexion exportiert: {filename}")
            print(f"📊 Enthält:")
            print(f"   • {len(reflection_notes)} Reflexions-Notizen")
            print(f"   • {total_tests} Test-Ergebnisse")
            print(f"   • {total_analyses} Bias-Analysen")
            
            # Kurze Zusammenfassung anzeigen
            print(f"\\n📋 WORKSHOP-ZUSAMMENFASSUNG:")
            print(f"=" * 40)
            
            for note in reflection_notes:
                print(f"\\n{note['category_title']}:")
                preview = note['content'][:100] + "..." if len(note['content']) > 100 else note['content']
                print(f"   {preview}")
    
    # Event-Bindings
    category_selector.observe(update_questions, names='value')
    save_button.on_click(save_reflection_note)
    export_button.on_click(export_workshop_reflection)
    
    # Initial setup
    if category_selector.options:
        category_selector.value = category_selector.options[0][1]
    
    return widgets.VBox([
        widgets.HTML("<h3>📝 Workshop-Reflexion</h3>"),
        widgets.HTML("<p>Strukturierte Dokumentation Ihrer Workshop-Erkenntnisse</p>"),
        category_selector,
        questions_display,
        notes_editor,
        widgets.HBox([save_button, export_button]),
        reflection_output
    ])

# Workshop-Reflexion anzeigen
workshop_reflection = create_workshop_reflection()
display(workshop_reflection)


In [ ]:
# 🎯 Workshop-Zusammenfassung & Nächste Schritte

print("🎓 MHDBDB Bias Detection Workshop - Abschluss")
print("=" * 55)
print()
print("📚 Was Sie gelernt haben:")
print("   ✓ Bias-Erkennung in LLM-Antworten zu historischen Begriffen")
print("   ✓ Kritische Analyse mittelalterlicher Terminologie")
print("   ✓ Prompt-Engineering für historische Forschung")
print("   ✓ Systematischen Vergleich verschiedener Ansätze")
print("   ✓ Strukturierte Reflexion von KI-Ergebnissen")
print()
print("🔧 Verwendete Tools:")
print("   • MLVoca.com - Kostenlose LLM API (TinyLlama, DeepSeek)")
print("   • Automatische Bias-Erkennung mit 5 Kategorien")
print("   • Prompt-Strategien-Vergleich")
print("   • Strukturierte Workshop-Reflexion")
print()
print("💡 Wichtige Erkenntnisse:")
print("   • LLMs neigen zu Anachronismen bei historischen Begriffen")
print("   • Prompt-Formulierung beeinflusst Bias-Neigung erheblich")
print("   • Kritische Quellenvalidierung bleibt unerlässlich")
print("   • Verschiedene Modelle zeigen unterschiedliche Bias-Muster")
print()
print("🚀 Nächste Schritte:")
print("   1. Exportieren Sie Ihre Workshop-Ergebnisse")
print("   2. Testen Sie weitere MHDBDB-Begriffe")
print("   3. Entwickeln Sie eigene Prompt-Strategien")
print("   4. Integrieren Sie Bias-Checks in Ihre Forschung")
print()
print("📖 Weiterführende Ressourcen:")
print("   • MHDBDB TEI Repository: github.com/DigitalHumanitiesCraft/mhdbdb-tei-only")
print("   • MLVoca.com Documentation: mlvoca.github.io/free-llm-api/")
print("   • Workshop-Beispielsammlung: siehe Workshop_Beispielsammlung.md")
print()
print("🙏 Vielen Dank für Ihre Teilnahme!")
print("   Feedback und Fragen gerne an das MHDBDB-Team")
print("=" * 55)

# Finaler Status-Check
def final_workshop_status():
    """Zeigt abschließenden Workshop-Status"""
    
    print("\\n📊 IHR WORKSHOP-STATUS:")
    print("-" * 30)
    
    # API-Status
    api_status = "✅ Verfügbar" if bias_lab.api_available else "❌ Nicht verfügbar"
    print(f"MLVoca API: {api_status}")
    
    # Tests durchgeführt
    test_count = len(bias_lab.test_results)
    print(f"Durchgeführte Tests: {test_count}")
    
    # Bias-Analysen
    analysis_count = len(bias_analyzer.analyses)
    print(f"Bias-Analysen: {analysis_count}")
    
    # Bias-Rate
    if bias_analyzer.get_summary_stats():
        bias_rate = bias_analyzer.get_summary_stats()['bias_rate']
        print(f"Durchschnittliche Bias-Rate: {bias_rate:.1f}%")
    
    # Empfehlungen
    print("\\n💡 EMPFEHLUNGEN:")
    if test_count == 0:
        print("   → Führen Sie einige Tests mit dem Prompt-Tool durch")
    elif test_count < 5:
        print("   → Testen Sie weitere Begriffe für umfassendere Erkenntnisse")
    else:
        print("   → Excellent! Nutzen Sie das Vergleichs-Tool für tiefere Analyse")
    
    if analysis_count == 0:
        print("   → Verwenden Sie das Bias-Analyse-Tool")
    
    print("   → Dokumentieren Sie Ihre Erkenntnisse im Reflexions-Tool")
    print("   → Exportieren Sie Ihre Ergebnisse für weitere Verwendung")

final_workshop_status()
